3) Se tiene un RDD con información de vuelos programados con la forma
(número de vuelo, código de aerolínea, código de aeropuerto de salida,
código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM,
fecha de llegada AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el
registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma
(número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM,
fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al estado,
podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado
no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso,
no contendrá información de la llegada. Se pide resolver utilizando PySpark:

a) Cuál es el aeropuerto con mayor tránsito.

In [96]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

aux = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej3a.csv')
programmedFlights = aux.map(lambda x: x.split(","))
programmedFlights = programmedFlights.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], x[2],
                         x[3], pd.to_datetime(x[4], 'coerce', format='%Y-%m-%d'), x[5],
                                                     pd.to_datetime(x[6], 'coerce', format='%Y-%m-%d'),
                                                     x[7])).cache()

#no encontre una mejor forma, el diccionario no funca porque pyspark no pasa por referencia las globales
leaveFlightsCount = programmedFlights.map(lambda x: (x[2], 1)).reduceByKey(lambda x,y: x + y)
arrivalFlightsCount = programmedFlights.map(lambda x: (x[3], 1)).reduceByKey(lambda x,y: x + y)

leaveFlightsCount.fullOuterJoin(arrivalFlightsCount).mapValues(lambda x: x[1] if (x[0] is None) else x[1] + x[0]) \
                .reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'PRKT'

b) Cuál es la aerolínea con mayor cantidad de vuelos.

In [97]:
programmedFlights.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x + y) \
                .reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'JA'

c) Cuál es la aerolínea con mayor cantidad de cancelaciones.

In [98]:
aux = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej3b.csv')
statusFlights = aux.map(lambda x: x.split(","))
statusFlights = statusFlights.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1],
                                             pd.to_datetime(x[2], 'coerce', format='%Y-%m-%d'), x[3],
                                             pd.to_datetime(x[4], 'coerce', format='%Y-%m-%d'),
                                             x[5], x[6])).cache()
statusFlights.filter(lambda x: x[6] == 'cancelado').map(lambda x: (x[1], 1)) \
            .reduceByKey(lambda x,y: x + y).reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'JA'

d) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.

In [99]:
flightDiff = programmedFlights.map(lambda x: (x[0], (x[4], x[5])))\
    .join(statusFlights.filter(lambda x: x[6] == 'activo').map(lambda x: (x[0], x[3])))\
    .mapValues(lambda x: (x[0][0], abs(pd.to_datetime(x[0][1]) - pd.to_datetime(x[1]))))

worstFlight = flightDiff.reduce(lambda x,y: x if (x[1][1] > y[1][1]) else y)

(worstFlight[0], worstFlight[1][0])

(284, Timestamp('2019-07-02 00:00:00'))

e) Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.

In [100]:
flightDiff = programmedFlights.map(lambda x: (x[0], (x[6], x[7])))\
    .join(statusFlights.filter(lambda x: x[6] == 'activo').map(lambda x: (x[0], x[5])))\
    .mapValues(lambda x: (x[0][0], abs(pd.to_datetime(x[0][1]) - pd.to_datetime(x[1]))))

worstFlight = flightDiff.reduce(lambda x,y: x if (x[1][1] > y[1][1]) else y)

(worstFlight[0], worstFlight[1][0])

(276, Timestamp('2019-07-07 00:00:00'))

f) Cuál es la aerolínea más puntual.

In [101]:
def time_to_seconds(x):
    (h, m, s) = x.split(':')
    result = int(h) * 3600 + int(m) * 60 + int(s)
    return result

diffFlights = programmedFlights.map(lambda x: (x[1], (time_to_seconds(x[5]), time_to_seconds(x[7]))))\
    .join(statusFlights.filter(lambda x: x[6] == 'activo').map(lambda x: (x[1], (time_to_seconds(x[3]), time_to_seconds(x[5])))))

seqOp = lambda acc,element: acc + abs(element[0][0] - element[1][0]) + abs(element[1][0] - element[1][1])
combOp = lambda acc1,acc2: acc1 + acc2

diffFlights = diffFlights.aggregateByKey(0, seqOp, combOp).cache()
diffFlights.reduce(lambda x,y: x if (x[1] < y[1]) else y)[0]

'AAR'

f) Cuál es el aeropuerto que registra mayor desviación con respecto a los horarios coordinados.

In [102]:
diffFlights.reduce(lambda x,y: x if (x[1] > y[1]) else y)[0]

'JA'